In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import colors
from matplotlib import animation
from matplotlib import rc
rc('animation', html='html5')

cmap = colors.ListedColormap(['White','Blue','Green','Red'])

# #normalizes colour range values 
n = colors.Normalize(vmin=0,vmax=3)

In [ ]:
class Animal:
    
    # requried variables: needed for subclasses
    location = [0,0]
    stepSize = 1
    mapSize = 0
    foodEaten = 0
    hunger = 0
    maxHunger = 0
    probRepro = 0
    sense = 1
    alive = True
    beStill = False
    mated = False
    
    def __init__(self, mapSize, stepSize=None, location=None, maxHunger=1000, age=0):
        
        if location == None:
            location = [np.random.randint(0, mapSize), np.random.randint(0, mapSize)]
        self.location = location
        
        self.steps = age
        self.mapSize = mapSize
        self.foodEaten = 0
        self.hunger = 0
        self.alive = True
        self.maxHunger = maxHunger
        
    def step(self, direct = None):
        self.hunger = self.hunger + 1
        self.steps = self.steps + 1
        #move once for every stepSize
        
        for i in range(0, self.stepSize):
            #one for each direction
            if(direct == None):
                direct = np.random.randint(0,8)
            
            # if the direction is 1,0,7 move x by +1
            if ((direct==0) or (direct==1) or (direct==7)):
                self.location[0] = self.location[0] + 1
                
            # if the direction is 1,2,3 move y by +1
            if ((direct==1) or (direct==2) or (direct==3)):
                self.location[1] = self.location[1] + 1
                
            # if the direction is 3,4,5 move x by -1
            if ((direct==3) or (direct==4) or (direct==5)):
                self.location[0] = self.location[0] - 1
                
            # if the direction is 5,6,7 move y by -1
            if ((direct==5) or (direct==6) or (direct==7)):
                self.location[1] = self.location[1] - 1
        self.locationCheck()
            
    # check if location needs to wrap
    def locationCheck(self):
        
        for i in range(0,2):
            if (self.location[i] >= self.mapSize):
                self.location[i] = self.location[i] - self.mapSize
            elif (self.location[i] < 0):
                self.location[i] = self.mapSize - abs(self.location[i])
    
    def vicinityCheck(self, animal2):
    
        a1X = self.location[0]
        a1Y = self.location[1]
        a2X = animal2.location[0]
        a2Y = animal2.location[1]
        sense = self.sense
        nearby = False

        for i in range(-sense, (sense+1)):
            if nearby == False:
                if (a1X == (a2X + i)):
                    for j in range(-sense, (sense+1)):
                        if nearby == False:
                            if (a1Y == (a2Y + j)):
                                nearby = True
        return nearby

In [ ]:
class Food:
    location = []
    eaten = False
    mapSize = 0
    
    def __init__(self, mapSize, location = None):
        
        if location == None:
            location = [np.random.randint(0, mapSize), np.random.randint(0, mapSize)]
        self.location = location
        
        self.mapSize = mapSize

In [ ]:
class Rabbit(Animal):
    
    probRepro = 0.5
    litter = 1                                     # Edit this value?
    species = 'Rabbit'
    eatMush = False
    sense = 3
    
    def step(self, foodArray = None):
        self.mated = False
        if(foodArray != None):
            super().step(self.hunt(foodArray))
        else:
            super().step()
    
    def reproduced(self, rabbit2):
        self.hunger = self.hunger * 1.25           # Edit this value?
        rabbit2.hunger = rabbit2.hunger * 1.25     # Edit this value?
        
    def interactRabbit(self, rabbit, animalArray):
        together = False
        if not rabbit.beStill:
            together = self.vicinityCheck(rabbit)
        if together:
            if self.mated == False and rabbit.mated == False:
                if ((np.random.rand() < self.probRepro)):
                    for i in range(0, self.litter):
                        self.reproduce(animalArray, rabbit)
                    self.reproduced(rabbit)
                    return True
        return False
        
    def interactMushroom(self, mushroom):
        together = False
        if not mushroom.eaten:
            together = self.vicinityCheck(mushroom)
        if together:
            mushroom.eaten = True
            self.hunger = self.hunger / 1.25       # Edit this value?
            return True
        return False
    
    def reproduce(self, animalArray, rabbit):
        # need to be 8 months to reproduce
        if self.steps > 7 and rabbit.steps > 7:
            x = self.location[0]
            y = self.location[1]
            animalArray.append(Rabbit(self.mapSize, stepSize=self.stepSize, location=[x,y]))
            animalArray[-1].step()
            self.mated = True
            rabbit.mated = True
            return True
        return False
    
    def hunt(self, foodArray):
        
        ax = self.location[0]
        ay = self.location[1]
        sense = self.sense
        
        goodX = []
        inRange = []
        directions = []
        
        for i in range(0, len(foodArray)):
            tempx = foodArray[i].location[0]
            if(tempx < (ax+sense) and (tempx > ax-sense)):
                goodX.append(i)
                
        if (len(goodX) == 0):
            return None
        
        for i in range(0, len(goodX)):
            tempy = foodArray[goodX[i]].location[1]
            if((tempy < (ay+sense)) and (tempy > (ay-sense))):
                inRange.append(goodX[i])
                
        if (len(inRange) == 0):
            return None
        
        steps = 100
        
        for i in range(0, len(inRange)):
            
            tempx = foodArray[inRange[i]].location[0]
            tempy = foodArray[inRange[i]].location[1]
            
            if(abs(ax-tempx) > abs(ay-tempy)):
                if(steps > abs(ax-tempx)):
                    steps = abs(ax-tempx)
            else:
                if(steps > abs(ax-tempx)):
                    steps = abs(ay-tempy)
        
        for i in range(0, len(inRange)):
            
            tempx = foodArray[inRange[i]].location[0]
            tempy = foodArray[inRange[i]].location[1]
            
            xdist = abs(ax-tempx)
            ydist = abs(ay-tempy)
            
            if(xdist == steps or ydist == steps):    #only the closest ones
                if(xdist < ydist):
                    #choose xdist
                    if((ax-tempx) < 0):
                        return 4
                    if((ax-tempx) > 0):
                        return 0
                elif(ydist < xdist):
                    #choose ydist
                    if ((ay-tempy) < 0):
                        return 6
                    if ((ay-tempy) > 0):
                        return 2
                else:
                    if (((ax-tempx) < 0) and ((ay-tempy) < 0)):
                        return 3
                    if (((ax-tempx) < 0) and ((ay-tempy) > 0)):
                        return 5
                    if (((ax-tempx) > 0) and ((ay-tempy) < 0)):
                        return 1
                    if (((ax-tempx) > 0) and ((ay-tempy) > 0)):
                        return 7

In [ ]:
class Fox(Animal):
    
    probRepro = 0.3
    litter = 1                                         # Edit this value?
    species = 'Fox'
    matedLast = 0
    sense = 3
    
    def step(self, foodArray = None):
        if self.mated == True:
            # 12 steps need to have occurred before mating again
            if self.steps - self.matedLast == 12:
                self.mated = False 
        if(foodArray != None):
            super().step(self.hunt(foodArray))
        else:
            super().step()
    
    def reproduced(self, fox):
        self.hunger = self.hunger * 1.25               # Edit this value?
        fox.hunger = fox.hunger * 1.25                 # Edit this value?
        
    def interactRabbit(self, rabbit):
        together = False
        if not rabbit.beStill:
            together = self.vicinityCheck(rabbit)
        if (together):
            rabbit.beStill = True
            self.hunger = self.hunger / 1.25 # Edit this value?
            return True
        return False
        
    def interactFox(self, fox, animalArray):
        together = False
        if not fox.beStill:
            together = self.vicinityCheck(fox)
        if together:
            if self.mated == False and fox.mated == False:
                if ((np.random.rand() < self.probRepro)):
                    for i in range(0, self.litter):
                        self.reproduce(animalArray, fox)
                    self.reproduced(fox)
                    return True
        return False
                    
    #add interact mushroom to allow for omnivourism  
    """Suggested edit to alter how much hunger a mushrooms satisfies to be less than a rabbit"""
    def interactMushroom(self, mushroom):
        together = False
        if not mushroom.eaten:
            together = self.vicinityCheck(mushroom)
        if together:
            mushroom.eaten = True
            self.hunger = self.hunger / 1.25
            return True
        return False
                
    def reproduce(self, animalArray, fox):
        # need to be 10 months to reproduce
        if self.steps > 9 and fox.steps > 9:
            x = self.location[0]
            y = self.location[1]
            animalArray.append(Fox(self.mapSize, stepSize=self.stepSize, location=[x,y]))
            animalArray[-1].step()
            self.mated = True
            self.matedLast = self.steps
            fox.mated = True
            fox.matedLast = fox.steps
            return True
        return False
    
    def hunt(self, foodArray):
        
        ax = self.location[0]
        ay = self.location[1]
        sense = self.sense
        
        goodX = []
        inRange = []
        directions = []
        
        for i in range(0, len(foodArray)):
            tempx = foodArray[i].location[0]
            if(tempx < (ax+sense) and (tempx > ax-sense)):
                goodX.append(i)
                
        if (len(goodX) == 0):
            return None
        
        for i in range(0, len(goodX)):
            tempy = foodArray[goodX[i]].location[1]
            if((tempy < (ay+sense)) and (tempy > (ay-sense))):
                inRange.append(goodX[i])
                
        if (len(inRange) == 0):
            return None
        
        steps = 100
        
        for i in range(0, len(inRange)):
            
            tempx = foodArray[inRange[i]].location[0]
            tempy = foodArray[inRange[i]].location[1]
            
            if(abs(ax-tempx) > abs(ay-tempy)):
                if(steps > abs(ax-tempx)):
                    steps = abs(ax-tempx)
            else:
                if(steps > abs(ax-tempx)):
                    steps = abs(ay-tempy)
        
        for i in range(0, len(inRange)):
            
            tempx = foodArray[inRange[i]].location[0]
            tempy = foodArray[inRange[i]].location[1]
            
            xdist = abs(ax-tempx)
            ydist = abs(ay-tempy)
            
            if(xdist == steps or ydist == steps):    #only the closest ones
                if(xdist < ydist):
                    #choose xdist
                    if((ax-tempx) < 0):
                        return 4
                    if((ax-tempx) > 0):
                        return 0
                elif(ydist < xdist):
                    #choose ydist
                    if ((ay-tempy) < 0):
                        return 6
                    if ((ay-tempy) > 0):
                        return 2
                else:
                    if (((ax-tempx) < 0) and ((ay-tempy) < 0)):
                        return 3
                    if (((ax-tempx) < 0) and ((ay-tempy) > 0)):
                        return 5
                    if (((ax-tempx) > 0) and ((ay-tempy) < 0)):
                        return 1
                    if (((ax-tempx) > 0) and ((ay-tempy) > 0)):
                        return 7

In [ ]:
class Mushroom(Food):
    
    probRepro = 0.05                                    # MANIPULATE THIS VALUE FOR EXPERIMENT
    probDecomp = 0.05                                   # MANIPULATE THIS VALUE FOR EXPERIMENT
    litter = 1                                          # Edit this value?
    species = 'Mushroom'
    
    def asexualReproduction(self, foodArray):
        if ((np.random.rand() < self.probRepro)):
            for i in range(0, self.litter):
                foodArray.append(Mushroom(self.mapSize))
                
    def decomposerSpawn(self, foodArray):
        if ((np.random.rand() < self.probDecomp)):
            for i in range(0, self.litter):
                foodArray.append(self)

In [ ]:
class Ecosystem:
    def __init__(self, rows, omni=None):
        self.mapSize = rows
        self.grid = np.zeros((rows, rows), dtype=int)
        self.foxes_array = []
        self.rabbits_array = []
        self.mush_array = []
        self.numFoxes = []
        self.numRabbits = []
        self.numMushrooms = []
        self.foxesDead = False
        self.rabbitsDead = False
        self.hunt=True
        
        if omni != None:
            self.omni = omni
        else :
            self.omni = False 
        
    # start the simulation with adults
    def createFoxes(self, numFoxes, fox_step_size, maxHunger=10, age=10):
        self.numFoxes.append(numFoxes)
        for i in range(numFoxes):
            fox = Fox(mapSize=self.mapSize, stepSize=fox_step_size, maxHunger=maxHunger, age=age)
            self.foxes_array.append(fox)
    
    def createRabbits(self, numRabbits, rabbit_step_size, maxHunger=10, age=8):
        self.numRabbits.append(numRabbits)
        for i in range(numRabbits):
            rabbit = Rabbit(mapSize=self.mapSize, stepSize=rabbit_step_size, maxHunger=maxHunger, age=age)
            self.rabbits_array.append(rabbit)
        
    def createMushrooms(self, numMushrooms):
        self.numMushrooms.append(numMushrooms)
        for i in range(numMushrooms):
            mush = Mushroom(mapSize=self.mapSize)
            self.mush_array.append(mush)
            
    def step(self):
        if (self.hunt):
            for i in range(len(self.foxes_array)):
                self.foxes_array[i].step((self.rabbits_array))
            for i in range(len(self.rabbits_array)):
                self.rabbits_array[i].step((self.mush_array))
        else:
            for i in range(len(self.foxes_array)):
                self.foxes_array[i].step()
            for i in range(len(self.rabbits_array)):
                self.rabbits_array[i].step()
            
    def mapToGrid(self):
        self.grid = np.zeros((self.mapSize, self.mapSize), dtype=int)
        currRabbits = len(self.rabbits_array)
        currFoxes = len(self.foxes_array)
        currMush = len(self.mush_array)

        for i in range(max(currRabbits, currFoxes, currMush)):
            # map a mushroom if its still not eaten
            if i < currMush:
                if not self.mush_array[i].eaten:
                    x = self.mush_array[i].location[0]
                    y = self.mush_array[i].location[1]
                    self.grid[x, y] = 1
        
            # map a rabbit to grid if its alive
            if i < currRabbits:
                if not self.rabbits_array[i].beStill:
                    x = self.rabbits_array[i].location[0]
                    y = self.rabbits_array[i].location[1]
                    self.grid[x, y] = 2
        
            # map a fox to grid if its alive
            if i < currFoxes:
                if not self.foxes_array[i].beStill:
                    x = self.foxes_array[i].location[0]
                    y = self.foxes_array[i].location[1]
                    self.grid[x, y] = 3
        
        return self.grid
    
    def plotGrid(self, grid):
        plt.imshow(grid[::-1],cmap=cmap, norm=n)
        
    def animate(self, maxFrames=200):
        fig = plt.figure()

        grid = self.mapToGrid()
        img = plt.imshow(grid[::-1],cmap=cmap,norm=n,animated=True)
        ims = []
        frames = 0
        # loop until a species is extinct
        while self.foxesDead == False and self.rabbitsDead == False:
            self.step()

            # check interactions
            self.checkInteractions()
            self.removeTheDead()
            
            # check population sizes
            self.numFoxes.append(len(self.foxes_array))
            self.numRabbits.append(len(self.rabbits_array))
            self.numMushrooms.append(len(self.mush_array))

            # plot stuffs
            grid = self.mapToGrid()
            img = plt.imshow(grid[::-1],cmap=cmap,norm=n, animated=True)
            ims.append([img])
            frames = frames + 1
            if frames == maxFrames:
                break

        return animation.ArtistAnimation(fig, ims, interval=200, blit=True,
                                        repeat_delay=1000)
    
    def checkInteractions(self):
        # only want to loop through existing animals / mushrooms
        currRabbits = len(self.rabbits_array)
        currFoxes = len(self.foxes_array)
        currMush = len(self.mush_array)
        
        for i in range(max(currFoxes, currRabbits, currMush)):
            # there are still foxes
            if i < currFoxes:
                fox = self.foxes_array[i]
                # check interactions with all foxes and rabbits
                for j in range(max(currFoxes, currRabbits)):
                    eatRabbit = False
                    if j != i and j < currFoxes:
                        # does the fox reproduce
                        fox.interactFox(self.foxes_array[j], self.foxes_array)
                    if j < currRabbits:
                        # does the fox eat a rabbit
                        eatRabbit = fox.interactRabbit(self.rabbits_array[j])
                    if j < currMush:
                        # does the fox eat a mushroom, if have not already eaten a rabbit
                        if self.omni == True:
                            if not eatRabbit:
                                fox.interactMushroom(self.mush_array[j])
            
            # there are still rabbits
            if i < currRabbits:
                rabbit = self.rabbits_array[i]
                # check interactions with all rabbits and mushrooms
                for j in range(max(currRabbits, currMush)):
                    if j != i and j < currRabbits:
                        # does the rabbit reproduce
                        rabbit.interactRabbit(self.rabbits_array[j], self.rabbits_array)
                    if j < currMush:
                        # does the rabbit eat a mushroom
                        rabbit.interactMushroom(self.mush_array[j])
            
            # there are still mushrooms
            if i < currMush:
                mushroom = self.mush_array[i]
                # mushrooms perform asexual reproduction
                mushroom.asexualReproduction(self.mush_array)
                
    def removeTheDead(self):
        # check if animals have died of starvation
        self.foxesDead = self.hungerCheck(self.foxes_array)
        self.rabbitsDead = self.hungerCheck(self.rabbits_array)
        
        # mushrooms decompose dead animals that die from starvation
        starved_foxes = [fox for fox in self.foxes_array if fox.beStill]
        starved_rabbits = [rabbit for rabbit in self.rabbits_array if rabbit.beStill]
        
        for i in range(max(len(starved_foxes), len(starved_rabbits))):
            # there are starved foxes
            if i < len(starved_foxes):
                x = starved_foxes[i].location[0]
                y = starved_foxes[i].location[1]
                decompMush = Mushroom(mapSize=self.mapSize, location=[x,y])
                decompMush.decomposerSpawn(self.mush_array) # probability check for decomposer to spawn
            #there are starved rabbits
            if i < len(starved_rabbits):
                x = starved_rabbits[i].location[0]
                y = starved_rabbits[i].location[1]
                decompMush = Mushroom(mapSize=self.mapSize, location=[x,y])
                decompMush.decomposerSpawn(self.mush_array) # probability check for decomposer to spawn

        # remove dead animals
        self.foxes_array = [ fox for fox in self.foxes_array if not fox.beStill]   
        self.rabbits_array = [ rabbit for rabbit in self.rabbits_array if not rabbit.beStill]
        
        self.foxesDead = True if len(self.foxes_array) == 0 else False
        self.rabbitsDead = True if len(self.rabbits_array) == 0 else False

        # remove mushrooms that have been eaten
        self.mush_array = [ mush for mush in self.mush_array if not mush.eaten]
                
    def hungerCheck(self, animalArray):
        
        everyoneDead = True
        if len(animalArray) == 0:
            return everyoneDead
        
        maxHunger = animalArray[0].maxHunger

        for i in range(0, len(animalArray)):

            hunger = animalArray[i].hunger

            if hunger > maxHunger:
                animalArray[i].beStill = True
            else:
                everyoneDead = False

        return everyoneDead

    def plotPopulationHist(self):
        x = range(len(self.numFoxes))
        
        plt.plot(x, self.numFoxes, label='Foxes', color='r')
        plt.plot(x, self.numRabbits, label='Rabbits', color='g')
        plt.plot(x, self.numMushrooms, label='Mushrooms', color='b')
        xl = plt.xlabel("Sample frames")
        yl = plt.ylabel("Population")
        t = plt.title("Population Growth")
        legend = plt.legend()

In [ ]:
## Declarations
rows=20

eco = Ecosystem(rows, omni=False)

#Population of Species
numFoxes=10
numRabbits=20
numMushrooms=50

rabbit_step_size=1
fox_step_size=1

## Initialise and Spawn Foxes & Rabbits
    
# Foxes
eco.createFoxes(numFoxes, fox_step_size, maxHunger=15)

# Rabbits
eco.createRabbits(numRabbits, rabbit_step_size, maxHunger=12)

# Mushrooms
eco.createMushrooms(numMushrooms)

# Show the initial state of the forest
data = eco.mapToGrid()
eco.plotGrid(data)

In [ ]:
maxFrames = 2000
anim = eco.animate(maxFrames)

In [ ]:
anim

In [ ]:
eco.plotPopulationHist()